In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


IMPORTING LIBRARIES

In [3]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.applications import VGG16

In [4]:
train_data = ImageDataGenerator()
test_data = ImageDataGenerator()

In [5]:
img_size=(224,224)
batch_size = 32

In [6]:
vgg=VGG16(weights='imagenet',include_top='False',input_shape=(224,224,3))

553467096/553467096 [==============================] - 13s 0us/step


In [7]:
for layer in vgg.layers:
  layer.trainable=False

Layers

In [19]:
x = Flatten()(vgg.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

Paths

In [11]:
sr_train_dir = '/content/drive/MyDrive/chest_xray/train'
sr_val_dir='/content/drive/MyDrive/chest_xray/val'
sr_test_dir='/content/drive/MyDrive/chest_xray/test'

In [12]:
sr_train_generator = train_data.flow_from_directory(
    sr_train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Found 5216 images belonging to 2 classes.


In [14]:
sr_test_generator = test_data.flow_from_directory(
    sr_test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['NORMAL','PNEUMONIA'],
    shuffle=False
)

Found 624 images belonging to 2 classes.


In [24]:
sr_val_generator = test_data.flow_from_directory(
    sr_val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Found 16 images belonging to 2 classes.


Model training

In [20]:
model_sr = Model(inputs=vgg.input, outputs=predictions)

In [21]:
model_sr.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
history_sr = model_sr.fit(sr_train_generator, validation_data=sr_val_generator, epochs=10)


KeyboardInterrupt: 

In [ ]:
test_loss_1, test_acc_1 = model_sr.evaluate(sr_test_generator)
print("Test Accuracy:", test_acc_1)

In [ ]:
model_sr.save("Srgan_pneumonia_model.h5")